In [26]:
import pandas as pd
import numpy as np

In [27]:
# Standardization formula: z = (x - mean) / standard_deviation

class Standardiser():
    def __init__(self) -> None:
        pass
    
    def fit(self, X):
        self.size = X.shape[1]
        
        self.mean = X.mean(axis=0)
        self.std = X.std(axis=0)
        return self     #returns self, which allows method chaining
    
    def transform(self,X):
        if X.shape[1] != self.size:
            raise Exception("Wrong array dimensions!")
        return (X-self.mean)/self.std
        

In [28]:
def shuffle_data(X, Y):

    data_num = np.arange(X.shape[0])
    np.random.shuffle(data_num)
    return X[data_num], Y[data_num]

def train_test_split(X, Y, test_size=0.5, shuffle=True):
    """
    Splits dataset into training and test sets.
    
    Parameters:
    ----------
    X : array-like
        Feature dataset.
    y : array-like
        Target values.
    test_size : float or int, default=0.5
        - If float (0 < test_size < 1), it represents the proportion of the dataset to include in the test split.
        - If int (1 <= test_size < len(y)), it represents the absolute number of test samples.
    shuffle : bool, default=True
        If True, shuffles data before splitting.
    
    Returns:
    -------
    X_train, X_test, y_train, y_test : array-like
        Split feature and target sets for training and testing."""
    
    if shuffle:
        X, Y = shuffle_data(X, Y)
    if test_size <1 :
        train_ratio = len(Y) - int(len(Y) *test_size)
        X_train, X_test = X[:train_ratio], X[train_ratio:]
        Y_train, Y_test = Y[:train_ratio], Y[train_ratio:]
        return X_train, X_test, Y_train, Y_test
    elif test_size in range(1,len(Y)):
        X_train, X_test = X[test_size:], X[:test_size]
        Y_train, Y_test = Y[test_size:], Y[:test_size]
        return X_train, X_test, Y_train, Y_test
    

def cross_val_split(X, Y, cv):
    l = len(X)
    size = int(l/cv)
    X_splits, Y_splits = [], []

    for i in range(cv):
        X_temp = X[size * i : size * (i + 1)]
        Y_temp = Y[size * i : size * (i + 1)]
        
        X_splits.append(X_temp)
        Y_splits.append(Y_temp)

    # Stack all subsets along a new axis
    X_split = np.stack(X_splits, axis=0)
    Y_split = np.stack(Y_splits, axis=0)

    return X_split, Y_split
    

In [29]:
class LWLR():
    def __init__(self,tau):
        self.tau = tau
  
    def fit(self,X, Y):
        self.dim = X.ndim
        self.X = self._add_ones(X)
        self.X_T = X.T
        self.Y = Y

    def predict(self, P):
        P = self._add_ones(P)
        out = []
        for p in P:
            # Vectorized calculation of weights
            W = np.exp(-np.sum((self.X - p) ** 2, axis=1) / (2 * self.tau ** 2))    #Do the sume because we have many predictors
            W_diag = np.diag(W)

            # Calculate theta using vectorized operations
            XT_W_X = self.X_T @ W_diag @ self.X
            XT_W_Y = self.X_T @ W_diag @ self.Y
            theta = np.linalg.solve(XT_W_X, XT_W_Y) #Solving XT_W_X @ theta = XT_W_Y is faster and more stable than calculating np.linalg.inv(XT_W_X) @ XT_W_Y

            # Make prediction
            out.append((theta @ p).squeeze())
        return np.array(out)

    def change_tau(self,tau):
        self.tau = tau

    def _add_ones(self, X):
        if type(X) != np.ndarray:
            X = np.array(X)
        return np.concatenate((np.ones((len(X),1)), X.reshape(len(X),-1)), axis=1)

In [30]:
#Read the data

dile_name = 'data-reg.csv'
raw_data = pd.read_csv(dile_name, header=None).to_numpy()

X = raw_data[:,:5]
Y = raw_data[:,-1]

print(f"Mean of all predictors{X.mean(axis=0)}")

scaler = Standardiser().fit(X)

X_scaled = scaler.transform(X)
print(f"Mean of all predictors standardised{X_scaled.mean(axis=0)}")

print(raw_data[1:5,:])


X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, shuffle=False)

X_split, Y_split = cross_val_split(X_train, Y_train, 4)



   
   


Mean of all predictors[0.20131508 8.97294864 0.13104092 0.11286003 0.22287212]
Mean of all predictors standardised[-2.53208760e-17  0.00000000e+00 -7.79103877e-18  4.28507132e-17
 -1.81790905e-17]
[[ 1.42724753e+00  1.49424745e+02  3.43088589e+00  4.66607724e+00
   9.00486546e+00  2.97000000e+01]
 [ 2.18171598e+00 -7.07277125e+01 -2.59226200e-01 -2.36445576e+00
   2.37749440e+00  1.78000000e+01]
 [ 5.95405824e+00  6.17248717e+02  1.08111101e+01  6.89822116e+00
   1.27327617e+01  2.68000000e+01]
 [ 4.22955892e+00  2.45741445e+02 -3.94933829e+00  5.22799998e+00
   2.79170509e+00  1.66000000e+01]]


In [31]:
np.sum(X_scaled,axis=1).shape

(342,)